## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [73]:
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice
from networkx import  minimum_spanning_tree, parse_edgelist, minimum_spanning_edges
import numpy as np

def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [125]:
def lower_bound_tsp(count, vertex_coordinates: List[Tuple[int,int]]) -> float:
    
    def get_graph():
        lines = []
        for i in range(1, len(vertex_coordinates) + 1): # Переводим заданный граф в формат, нужный стандартным функциям
            for j in range(1, len(vertex_coordinates) + 1):
                lines.append( str(i) + " " + str(j) + " {'weight':" + 
                         str(euclidean_distance(vertex_coordinates[i - 1], vertex_coordinates[j - 1])) + '}')
        Gr = parse_edgelist(lines, nodetype = int) # Граф, к которому уже применимы стандартные функции.
        return Gr
    
    
    def get_tree_weight():
        T = minimum_spanning_edges(G)
        weight = 0
        for edge in T: # Вычисляем вес минимального остовного дерева, складывая веса всех его рёбер.
            weight += euclidean_distance(vertex_coordinates[edge[0] - 1], vertex_coordinates[edge[1] - 1])
        return(weight)
    

    l = []
    size = len(vertex_coordinates)
    G = get_graph() # Считываем граф
    best_weight = get_tree_weight() # Вес остовного дерева.
    y = np.zeros(size) # "Потенциалы".
    two = np.ones_like(2)
    alpha = 2
    counts = np.zeros(size) # Степени вершин в дереве.
    weight = best_weight
    U = 2 * best_weight # Верхняя оценка. Как известно, длина оптимума не превосходит удвоенного веса остовного дерева.
    t = 100
    for times in range(0, count): # Количество шагов алгоритма.

        counts = np.zeros(size)
        for (fr, to) in G.edges() : # Подсчитываем степени вершин дерева.
            counts[fr - 1] += 1
            
        t = alpha *(U - weight)/(( two - counts)*( two - counts)).sum() # Пересчитываем "шаг" алгоритма.
        alpha *= 2/3 # Уменьшаем параметр каким-то произвольным образом.
        
        for i in range(0, size): # Пересчитываем потенциалы.
            y[i] += t * (2 - counts[i])
        
        for fr in range(1, size + 1): # Обновляем веса рёбер.
            for to in range(1, size + 1):
                if G.has_edge(fr, to):
                    G[fr][to]['weight'] -= y[fr-1]
        
        weight = get_tree_weight() + 2 * y.sum() # Оценка.
        
        if(weight > best_weight):
            best_weight = weight
   
    return(best_weight)

In [129]:
import time
from os.path import exists

def run_all():
    instance_filenames = [['d198.tsp', 15], ['d493.tsp', 15], ['d657.tsp', 15], 
                          ['d2103.tsp', 3], ['pr107.tsp', 15], ['pr152.tsp',15],
                          ['pr439.tsp', 15]]
    for filename, count in instance_filenames:
        if not exists(filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance(filename)
        print('Instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        bound = lower_bound_tsp(count, instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))

In [130]:
run_all()

Instance d198.tsp… done in 5.9 seconds with lower bound 11767
Instance d493.tsp… done in 3.6e+01 seconds with lower bound 29284
Instance d657.tsp… done in 5.3e+01 seconds with lower bound 42490
Instance d2103.tsp… done in 2.8e+02 seconds with lower bound 76300
Instance pr107.tsp… done in 2.1 seconds with lower bound 34757
Instance pr152.tsp… done in 3.8 seconds with lower bound 59168
Instance pr439.tsp… done in 3.2e+01 seconds with lower bound 92193


## Выводы
Запишите здесь полученные результаты относительно погрешностей алгоритмов NN и NI.

Из предыдущего задания 3-1 имеем маршруты:

1

Solving instance d198.tsp... with tour length 18506 using NN and ... with tour length 19944 using NI

Instance d198.tsp... done ... with lower bound 11767

Найденные значения на около 57% и 69%  для NN и NI соответственно больше оценки


2 

Solving instance d493.tsp... with tour length 43699 using NN and ... s with tour length 48762 using NI

Instance d493.tsp ... done ... with lower bound 29284

Найденные значения на около 50% и  65% для NN и NI соответственно больше оценки

3

Solving instance d657.tsp... with tour length 65173 using NN and ... with tour length 73468 using NI

Instance d657.tsp… done ... with lower bound 42490

Найденные значения на около 53% и 72%  для NN и NI соответственно больше оценки

4

Solving instance d2103.tsp… done...with tour length 94185 using NN and ...with tour length 137254 using NI

Instance d2103.tsp… done ... seconds with lower bound 76300

Найденные значения на около 23% и 80%  для NN и NI соответственно больше оценки 

5

Solving instance pr107.tsp… done... with tour length 51452 using NN and...with tour length 54492 using NI

Instance pr107.tsp… done... with lower bound 34757

Найденные значения на около 48% и 57%  для NN и NI соответственно больше оценки

6

Solving instance pr152.tsp… done... with tour length 88823 using NN and ...with tour length 106938 using NI

Instance pr152.tsp… done ... with lower bound 59168

Найденные значения на около 50% и 81%  для NN и NI соответственно больше оценки

7

Solving instance pr439.tsp… done... with tour length 131294 using NN and ... with tour length 173907 using NI

Instance pr439.tsp… done... with lower bound 92193

Найденные значения на около 42% и 89%  для NN и NI соответственно больше оценки


**Вывод:** на данных примерах жадные алгоритмы ошибаются в среднем на 50-70%, для этих примеров ошибка никогда не превышала более чем вдвое нижнюю оценку. 